In [1]:
from matplotlib import pyplot as plt
import math, os, re, time, random, string
import numpy as np, pandas as pd, seaborn as sns
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

from collections import defaultdict
import wordcloud

import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import *

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('./raw_data/reddit_train.csv', encoding='latin-1')
test = pd.read_csv('./raw_data/reddit_test.csv', encoding='latin-1')

In [3]:
def features(df):
    df['word count'] = df['BODY'].apply(lambda x: len(x.split())) #count number of words in text
    df['unique word count'] = df['BODY'].apply(lambda x: len(set(x.split()))) #count unique words
    df['char count'] = df['BODY'].apply(lambda x: len(x)) #count number of characters
    df['url count'] = df['BODY'].apply(lambda x: len([i for i in str(x) if i == 'http'])) #count number of urls
    df['stopword count'] = df['BODY'].apply(lambda x: len([i for i in x.lower().split() if i in wordcloud.STOPWORDS]))
    df['punctuation count'] = df['BODY'].apply(lambda x: len([i for i in str(x) if i in string.punctuation]))
    
features(df)
features(test)

In [4]:
df.columns

Index(['Unnamed: 0', 'X', 'BODY', 'REMOVED', 'word count', 'unique word count',
       'char count', 'url count', 'stopword count', 'punctuation count'],
      dtype='object')

In [5]:
def clean_text(text):
    '''Make text lowercase, remove text in square bracketsremove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

stop_words = stopwords.words('english')+['u', 'im', 'c']

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text 

stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

def preprocess(df):
    df = df.drop(['Unnamed: 0', 'X'], axis=1)
    df['cleaned'] = df['BODY'].apply(clean_text)
    df['stop_removed'] = df['cleaned'].apply(remove_stopwords)
    df['stemmed_no_stop'] = df['stop_removed'].apply(stemm_text)
    df['stemmed_with_stop'] = df['cleaned'].apply(stemm_text)
    df = df.dropna()
    return df

In [6]:
df = preprocess(df)
test = preprocess(test)

In [7]:
df.to_csv('./raw_data/train_processed.csv', index=False)
test.to_csv('./raw_data/test_processed.csv', index=False)

In [8]:
df.columns

Index(['BODY', 'REMOVED', 'word count', 'unique word count', 'char count',
       'url count', 'stopword count', 'punctuation count', 'cleaned',
       'stop_removed', 'stemmed_no_stop', 'stemmed_with_stop'],
      dtype='object')